In [3]:
import h5py as h5

f = 'test_data/COMPAS_Output/COMPAS_Output.h5'
d = h5.File(f)
d

<HDF5 file "COMPAS_Output.h5" (mode r)>

In [5]:
d.keys()

<KeysViewHDF5 ['BSE_Supernovae', 'BSE_System_Parameters', 'Run_Details']>

In [14]:
for k in d.keys():
    print(f"---{k}---")
    for subk in d[k].keys():
        print(subk, d[k][subk][:])
    

---BSE_Supernovae---
Applied_Kick_Magnitude(SN) [220.96978339  51.99531541  34.52438666 237.41422999 103.2534077 ]
ComponentSpeed(CP) [ 5.66879489  3.65123102 13.7279931  22.49346549  1.18842606]
ComponentSpeed(SN) [224.46000644  53.45222381  36.49418138 233.3283399  103.47373601]
Drawn_Kick_Magnitude(SN) [245.07587238  51.99531541  34.52438666 271.24535535 239.66005919]
Eccentricity [1082.60375248  168.63815692    4.07225386  159.7918011   596.73945304]
Eccentricity<SN [0. 0. 0. 0. 0.]
Experienced_RLOF(SN) [0 0 0 0 0]
Fallback_Fraction(SN) [0.09836174 0.         0.         0.12472518 0.56916723]
Is_Hydrogen_Poor(SN) [0 0 0 0 1]
Kick_Magnitude(uK) [22.74291281  8.55926661  1.20453681  8.39139047 39.58337557]
MT_Donor_Hist(SN) [b'NA              ' b'NA              ' b'NA              '
 b'NA              ' b'NA              ']
Mass(CP) [ 8.1120541   4.62695867  4.3519377   3.16700588 10.33895292]
Mass(SN) [2.00518008 1.26       1.26       2.23227349 6.07713963]
Mass_CO_Core@CO(SN) [3.7

In [2]:
!pip install h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.9 MB/s eta 0:00:0000:0100:01
